# Daily Job to Get PPV Values

> See V2__ SnowFlake Data for latest version

In [ ]:
import os
import requests
import os.path
import pandas as pd
import csv
import json
import pytz
import pygsheets

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from datetime import datetime, timedelta

headers = {
    'X-PW-AccessToken':os.environ["COPPER_API_KEY"],
    'X-PW-Application':'developer_api',
    'X-PW-UserEmail':'cooper@rehabpath.com',
    'Content-Type':'application/json'
}

def refresh_custom_fields(return_results = False):
    """
    Function to refresh the custom_field_json file and related dictionary. Option to 
    return resulting dictionary:

    return_results: True to return results, False to just update local file
    """
    # Fetching custom fields:
    list_fields = requests.get('https://api.copper.com/developer_api/v1/custom_field_definitions',headers=headers)
    
    # Was the API request a success?
    if list_fields.status_code != 200:
        print('Error fetching custom fields.')
        if return_results:
            print('Returning previous version.')
            with open('custom_fields.json', encoding="utf-8") as f:
                return json.load(f)
    
    # Obtaining response text and initializing dictionary:
    list_fields_json = list_fields.json()
    custom_field_dict = {}

    for item in list_fields_json:
        item_dict = {}
        for sub_item in ['name','data_type','available_on','is_filterable']:
            item_dict[sub_item] = item[sub_item] 

        item_options = {}

        if 'options' in item:
            for sub_item in item['options']:
                sub_item_name = sub_item['name']
                sub_item_id = sub_item['id']

                item_options[sub_item_id] = sub_item_name
        
            item_dict['options'] = item_options
        item_id = item['id']
        custom_field_dict[item_id] = item_dict
    
    # Updating local file:
    with open('custom_fields.json', 'w') as file:
        json.dump(custom_field_dict, file, indent=4)
    
    #Option to return results or just update local file:
    if return_results == True:
        return custom_field_dict
    else:
        print('Successfully Updated File!')
        pass

custom_fields = refresh_custom_fields(True)

def get_cf_info(cf_id,cf_info):
    """
    Function to get the custome field information based on the field id.

    cf_id (int or string) is the id_number of the custome field
    cf_info can be a single string or list of strings like 'name' or ['name','data_type','is_filterable']
    """

    if isinstance(cf_id,str):
        cf_id = int(cf_id)      

    # Check if cf_info item(s) are valid
    if isinstance(cf_info, list):
        for item_name in cf_info:
            # Remove faulty items:
            if item_name not in ['name','data_type','available_on','is_filterable','options']:
                cf_info.pop(item_name)
                print(f'Invalid cf_info: {cf_info}')
            
            # Cancel function if no valid items
            if len(cf_info) == 0:
                pass
    elif cf_info not in ['name','data_type','available_on','is_filterable','options']:
        print(f'Invalid cf_info: {cf_info}'); pass

    if isinstance(cf_info,list):
        return_list = []
        for item_name in cf_info:
            value = custom_fields[cf_id].get(item_name)
            return_list.append(value)
        return return_list
    else:
        value = custom_fields[cf_id].get(cf_info)
        return value

def get_cf_options(cf_id):
    return custom_fields.get(cf_id).get('options')

def cf_option_name(cf_id,option_id):
    return get_cf_options(cf_id).get(option_id)

def reformat_company_data(company_data):
    """
    Function to reformat the company data recieved from Copper in their JSON response
    """

    native_items = ['id', 'name', 'address', 'assignee_id', 'contact_type_id']

    output_dict = {}

    for item in native_items:
        output_dict[item] = company_data.get(item, None)
    
    custom_field_data = company_data['custom_fields']

    for dict_item in custom_field_data:
        item_id = dict_item['custom_field_definition_id']
        item_name = custom_fields_dict.get(item_id, None)

        if item_name not in Custom_Fields or item_name is None:
            continue
        
        if item_name is not None:
            item_value = dict_item['value']
            output_dict[item_name] = item_value
    
    return output_dict

def write_to_gsheet(service_file_path, spreadsheet_id, sheet_name, data_df):
    """
    this function takes data_df and writes it under spreadsheet_id
    and sheet_name using your credentials under service_file_path
    """
    gc = pygsheets.authorize(service_file=service_file_path)
    sh = gc.open_by_key(spreadsheet_id)
    try:
        sh.add_worksheet(sheet_name)
    except:
        pass
    wks_write = sh.worksheet_by_title(sheet_name)
    wks_write.clear('A1',None,'*')
    wks_write.set_dataframe(data_df, (1,1), encoding='utf-8', fit=True)
    wks_write.frozen_rows = 1


SCOPES = ["https://www.googleapis.com/auth/spreadsheets.readonly"]

def authorize_gspread(credentials_file, token_file, scopes):
    creds = None

    if os.path.exists(token_file):
        creds = Credentials.from_authorized_user_file(token_file, scopes)

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(credentials_file, scopes)
            creds = flow.run_local_server(port=0)

        with open(token_file, "w") as token:
            token.write(creds.to_json())

    return creds


def read_google_sheets(spreadsheet_id, range_name, credentials_file, token_file, scopes):
    creds = authorize_gspread(credentials_file, token_file, scopes)

    try:
        service = build("sheets", "v4", credentials=creds)

        sheet = service.spreadsheets()
        resulting_sheet = sheet.values().get(spreadsheetId=spreadsheet_id, range=range_name).execute()
        values = resulting_sheet.get("values", [])

        if not values:
            return None

        return values

    except HttpError as err:
        return None


def get_data(spreadsheet_id,range_name,credentials_file = "credentials.json",token_file = "token.json"):
    """
    Function to fetch data from Google Sheets.
    """
    result = read_google_sheets(spreadsheet_id, range_name, credentials_file, token_file, SCOPES)
    
    if result is not None:
        results = pd.DataFrame(result[1:], columns=result[0])
        return results
    else:
        print("Failed to retrieve data from Google Sheets.")
        pass

custom_fields_dict = {}

for item in custom_fields.keys():
    item_name = custom_fields[item].get('name')
    custom_fields_dict[item] = item_name

custom_fields_df = pd.DataFrame(list(custom_fields_dict.items()),columns = ['id','name'])

custom_fields_list = list(custom_fields_df['name'])

### Getting Data from Dave's Sheet:

In [ ]:
ppv_data = get_data("1GpB_tmHJvwc-Ie7WGT0_VAX1bh7ESKoiBYkGvQSgrFU","PPVs_PIVOT!A2:Ak")
ppv_data.rename(columns={'Page path without query string':'Page'},inplace=True)
ppv_data = ppv_data[ppv_data['Page']!= 'Grand Total']

### Getting Go Live Dates:

In [ ]:
Custom_Fields = ['Page','GoLive Date','Profile Slug']

In [ ]:
page = 1
combined_results = []
total_pages = 2

while page <= total_pages:
    page_params = {
        "page_size": 100,
        "page_number": page,
        "custom_fields": [
        {
            "custom_field_definition_id": 588393,
            "value": [1806073, 1806063,1806064,1824709,1806067]
        }
        ]}
    result = requests.post('https://api.copper.com/developer_api/v1/companies/search',headers=headers,json=page_params)
    
    if result.status_code == 200:
        total_pages = (int(result.headers['X-PW-TOTAL'])//100)+1
        print(f"Getting page {page} of {total_pages}.")
        result_json = result.json()
        combined_results.extend(result_json)
        page +=1

    else:
        print(f"Issue with page {page}. Stopping.")
        break

Getting page 1 of 3.
Getting page 2 of 3.
Getting page 3 of 3.


In [ ]:
cleaned_data = []

for idx in range(len(combined_results)):
    results = reformat_company_data(combined_results[idx])
    cleaned_data.append(results)

cleaned_data_df = pd.DataFrame(cleaned_data)
list_slugs = list(set(cleaned_data_df['Profile Slug']))
cleaned_data_df.drop(columns=['id','name','address','assignee_id','contact_type_id','Profile Slug'],inplace=True)

In [ ]:
ct_timezone = pytz.timezone('US/Central')
cleaned_data_df['GoLive Date'] = cleaned_data_df['GoLive Date'].apply(lambda x: pd.to_datetime(x, unit='s'))

### Getting Views from Snowflake

In [ ]:
yesterday = datetime.now(ct_timezone)-timedelta(1)
search_year = yesterday.year
search_month = yesterday.month

In [ ]:
views_df = _deepnote_execute_sql('SELECT * \nFROM RP_PROD_DL."CLEAN_TABLES_GA4".CENTER_DAILY_VIEWS \nWHERE YEAR(AS_OF_DATE) = {{search_year}} AND MONTH(AS_OF_DATE) = {{search_month}};', 'SQL_5E90C4C2_A521_4993_A1E9_D77837A4FD79', audit_sql_comment='', sql_cache_mode='cache_disabled')
views_df

,as_of_date,slug,property_id,region,city,number_of_views
0,2024-01-04,britelife-recovery,278764105,Florida,Orlando,4
1,2024-01-04,browse,278764105,Connecticut,Brookfield,6
2,2024-01-04,browse,278764105,Ohio,Springfield,6
3,2024-01-04,chabad-california,278764105,California,Los Angeles,2
4,2024-01-04,cura-west-colorado,278764105,Florida,Doral,2
...,...,...,...,...,...,...
32373,2024-01-07,the-ohana-hawaii,278764105,Florida,Miami,1
32374,2024-01-05,peaks-recovery,278764105,Texas,(not set),1
32375,2024-01-05,silicon-beach-womens-residence-los-angeles-cal...,278764105,Oklahoma,Oklahoma City,1
32376,2024-01-05,silver-hill-new-canaan-connecticut,278764105,Maryland,Pikesville,1


In [ ]:
views_df['as_of_date'] = views_df['as_of_date'].astype(str)
filtered_views = views_df[views_df['slug'].isin(list_slugs)][['as_of_date', 'slug', 'number_of_views']]
pivoted_views = filtered_views.groupby(['slug', 'as_of_date'])['number_of_views'].sum().unstack().reset_index()
pivoted_views

as_of_date,slug,2024-01-01,2024-01-02,2024-01-03,2024-01-04,2024-01-05,2024-01-06,2024-01-07
0,1000-islands,28.0,18.0,18.0,12.0,10.0,13.0,1.0
1,90210-recovery-california,5.0,2.0,4.0,4.0,2.0,6.0,1.0
2,abhasa-rehabilitation-india,15.0,11.0,7.0,8.0,7.0,12.0,6.0
3,abhasa-rehabilitation-thondamuthur-india,8.0,6.0,3.0,6.0,1.0,12.0,3.0
4,abhasa-wellness-retreat,7.0,5.0,7.0,2.0,3.0,13.0,4.0
...,...,...,...,...,...,...,...,...
248,wilmington-treatment-center-north-carolina,1.0,3.0,NaN,2.0,2.0,3.0,NaN
249,wish-recovery-california,21.0,13.0,16.0,17.0,17.0,33.0,15.0
250,within-center-austin-texas,4.0,3.0,11.0,13.0,8.0,8.0,1.0
251,zeus-rehab-warsaw-poland,5.0,1.0,2.0,3.0,7.0,13.0,1.0


### Removing PPVs Before GoLive Dates

In [ ]:
merged_df = pd.merge(ppv_data, cleaned_data_df, on='Page', how='left')
list_cols = merged_df.columns[1:-1]

In [ ]:
list_cols

Index(['2024-01-01', '2024-01-02', '2024-01-03', '2024-01-04', '2024-01-05',
       '2024-01-06'],
      dtype='object')

In [ ]:
merged_df['GoLive Date'] = pd.to_datetime(merged_df['GoLive Date'])

for index, row in merged_df.iterrows():
    go_live_date = row['GoLive Date']
    if pd.isnull(go_live_date):
        # If GoLive Date is missing, set all date columns to 0
        merged_df.loc[index, list_cols] = 0
    else:
        for col in list_cols:
            col_date = pd.to_datetime(col)+timedelta(1)
            if go_live_date > col_date:
                merged_df.at[index, col] = 0

In [ ]:
def ensure_32_columns(df):
    current_column_count = df.shape[1]
    columns_to_add = 33 - current_column_count

    for i in range(columns_to_add):
        df[f'Empty Column {current_column_count + i + 1}'] = pd.NA

    return df


merged_df = ensure_32_columns(merged_df)
merged_df = merged_df.where(pd.notnull(merged_df), '')

In [ ]:
finished_data = merged_df.drop(columns=['GoLive Date'])
finished_data

,Page,2024-01-01,2024-01-02,2024-01-03,2024-01-04,2024-01-05,2024-01-06,Empty Column 9,Empty Column 10,Empty Column 11,...,Empty Column 24,Empty Column 25,Empty Column 26,Empty Column 27,Empty Column 28,Empty Column 29,Empty Column 30,Empty Column 31,Empty Column 32,Empty Column 33
0,/1000-islands/,28,18,18,12,10,13,,,,...,,,,,,,,,,
1,/90210-recovery-california/,5,2,4,4,2,6,,,,...,,,,,,,,,,
2,/abhasa-rehabilitation-india/,15,11,7,8,7,12,,,,...,,,,,,,,,,
3,/abhasa-rehabilitation-thondamuthur-india/,8,6,3,6,1,11,,,,...,,,,,,,,,,
4,/abhasa-wellness-retreat/,7,5,7,2,3,12,,,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,/wilmington-treatment-center-north-carolina/,1,3,,2,2,3,,,,...,,,,,,,,,,
249,/wish-recovery-california/,21,13,16,17,17,33,,,,...,,,,,,,,,,
250,/within-center-austin-texas/,4,3,11,13,8,8,,,,...,,,,,,,,,,
251,/zeus-rehab-warsaw-poland/,0,0,0,0,7,13,,,,...,,,,,,,,,,


### Send Data to ADPB Sheet

In [ ]:
#write_to_gsheet('/work/keveldata-5f76a71e9aaf.json','1G5Mo3wrT_SrbFt5lyjKDne4C-Hj0cfjxCaW9mGPLQXU',"PPV Data",finished_data)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=6d4ba941-f702-478c-87bf-a3e3e8035885' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>